# Image Classification With Transfer Learning Using Keras

Import libraries

In [ ]:
import tensorflow as tf
import tarfile
import urllib
import os
import numpy as np
import pets

from matplotlib import pyplot as plt
%matplotlib inline

data_dir = 'data/'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

print('TensorFlow version:', tf.__version__)
print('Is using GPU?', tf.test.is_gpu_available())

# Cats vs Dogs Dataset

http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz

http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

In [ ]:
pets.download_and_extract('data', '.')

In [ ]:
class_to_index = {'cat': 0, 'dog': 1}
index_to_class = {0: 'cat', 1: 'dog'}

train_annot, count_train = pets.cats_vs_dogs_annotations('data/annotations/trainval.txt')
test_annot, count_test = pets.cats_vs_dogs_annotations('data/annotations/test.txt')

print('Training examples', count_train)
print('Test examples', count_test)

In [ ]:
test_annot

# Get Random Batch

In [ ]:
image_dir = 'data/images/'

def get_random_batch(annot, batch_size=4):
    all_keys = list(annot.keys())
    total_examples = len(all_keys)
    
    indices = np.random.choice(range(total_examples), batch_size, replace=False)
    
    x = np.zeros((batch_size, 128, 128, 3)) # shape of the image of the datasets limited by Keras pre-trained weights
    y = np.zeros((batch_size, 1))
    images = []
    
    for i, index in enumerate(indices):
        image_path = os.path.join(image_dir, all_keys[index])
        image = tf.keras.preprocessing.image.load_img(
            image_path, target_size=(128,128)
        )
        arr = tf.keras.preprocessing.image.img_to_array(image)
        arr = tf.keras.applications.mobilenet_v2.preprocess_input(arr)
        arr = np.expand_dims(arr, axis=0)
        x[i] = arr
        y[i] = class_to_index[annot[all_keys[index]]]
        images.append(image)
    return x, y, images

In [ ]:
x, y, images = get_random_batch(train_annot, batch_size=8)
pets.display_examples(x, y, y, images, index_to_class).show()

# Create Model

In [ ]:
mnet = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top=False, pooling='avg',
    weights='imagenet', input_shape=(128,128,3)
)
mnet.summary()

In [ ]:
def create_model():
    model = tf.keras.models.Sequential([
        mnet,
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    model.layers[0].trainable = False  # For don't change the pretrained model and avoid that the process of train only pass in the last layer
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model
model = create_model()
model.summary()

# Model Training

In [ ]:
def data_generator(batch_size, annot):
    while True:
        x, y, _ = get_random_batch(annot, batch_size)
        yield (x, y)

In [ ]:
batch_size = 32 # For the model training
steps_per_epoch = int(count_train / batch_size)
validation_steps = int(count_test/batch_size)

print('Training steps:', steps_per_epoch)
print('Validation steps:', validation_steps)

In [ ]:
%%time

_ = model.fit_generator(
    data_generator(batch_size, train_annot),
    validation_data = data_generator(batch_size, test_annot),
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=1
)

# Predictions

In [ ]:
x, y, images = get_random_batch(test_annot, batch_size=8)
preds = model.predict(x)

In [ ]:
pets.display_examples(x,y,preds,images,index_to_class).show()